# Cira playground

In [1]:
import cira

In [2]:
cira.auth.KEY_FILE = "../alpc_key.json"

assert cira.auth.check_keys(), "the set keys dose not work"

In [3]:
portfolio = cira.Portfolio()
stock_exchange = cira.StockExchange()
crypto_exchange = cira.StockExchange()

In [4]:
cry = cira.Cryptocurrency("ETH/USD")    
print(cry,cry.current_price())

spy = cira.Stock("SPY")
print(spy, spy.current_price())

ETH/USD 0.0
SPY 456.95


In [5]:
print(portfolio.cash())
pos = cira.Position("GOOGL")
print(pos, pos.quantity())

17597.6
GOOGL 1360


## ML example 

In [14]:
from datetime import datetime

cry = cira.Cryptocurrency("ETH/USD")    

start = datetime(2019, 9, 1)
end = datetime(2022, 9, 1)

data = cry.historical_data_df(start_date=start, end_date=end)
data.head()

open    high     low   close  volume  \
symbol  timestamp                                                           
ETH/USD 2020-04-05 05:00:00+00:00  143.71  143.71  143.71  143.71   0.018   
        2020-05-04 05:00:00+00:00  205.10  205.10  202.81  202.81  20.933   
        2020-05-22 05:00:00+00:00  199.34  209.84  199.34  209.84   5.519   
        2020-05-23 05:00:00+00:00  208.28  208.60  206.02  207.76   3.484   
        2020-05-24 05:00:00+00:00  210.47  210.47  201.42  201.42  11.933   

                                   trade_count        vwap  
symbol  timestamp                                           
ETH/USD 2020-04-05 05:00:00+00:00          3.0  143.710000  
        2020-05-04 05:00:00+00:00          2.0  203.961837  
        2020-05-22 05:00:00+00:00         13.0  206.092923  
        2020-05-23 05:00:00+00:00          6.0  207.307417  
        2020-05-24 05:00:00+00:00          5.0  208.982478

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


y = data[['close']]
X = data[['open', 'high', 'low']]

window_size = 100
y_preds = []
actual_prices = []

for i in range(len(X) - window_size):
    X_window = X.iloc[i:i + window_size]
    y_window = y.iloc[i:i + window_size]
    
    model = LinearRegression()
    model.fit(X_window.to_numpy(), y_window.to_numpy())
    
    next_day_features = X.iloc[i + window_size]
    next_day_pred = model.predict([next_day_features])[0]
    
    y_preds.append(next_day_pred)
    actual_prices.append(y.iloc[i + window_size])

mse = mean_squared_error(actual_prices, y_preds)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 2585.1473869405463


In [22]:


def run(y_preds:list, cash:float, shares:int) -> float:
    for i in range(len(y_preds) - 1):
        if y_preds[i + 1] > y_preds[i] and cash >= y_preds[i + 1]: 
            cash -= y_preds[i + 1]
            shares += 1
            #print(f"Buy 1 share at price: {y_preds[i + 1]}")
        
        elif y_preds[i + 1] < y_preds[i] and shares > 0: 
            cash += y_preds[i + 1]
            shares -= 1
            #print(f"Sell 1 share at price: {y_preds[i + 1]}")

    final_cash = cash + shares * y_preds[-1]
    return final_cash[0]

cash = portfolio.cash()  # Initial capital
shares = portfolio.position_in(str(cry)).quantity()  # Number of shares held
print("predicted profit/loss:", f"{run(y_preds, cash, shares) - portfolio.cash():.1f} USD$")


predicted profit/loss: 23423.7 USD$
